In [1]:
import tkinter as tk
from tkinter import ttk
from tkinter import messagebox
import mysql.connector as sql

All Queries

In [2]:
class SwiggyApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Swiggy Restaurant Data")

        # Attempt to connect to the MySQL database
        try:
            self.mydb = sql.connect(
                host="192.168.194.204",
                user="sagnik",
                passwd="remote",
                database="swiggy_python",
                port="3306"
            )
            self.mycursor = self.mydb.cursor()
        except sql.Error as e:
            messagebox.showerror("Database Error", f"Failed to connect to database: {e}")
            self.root.quit()
            return

        # Create the main frame and label
        self.main_frame = tk.Frame(self.root)
        self.main_frame.pack(padx=10, pady=10)

        tk.Label(self.main_frame, text="Select a query:").pack()

        # Add buttons for each query
        tk.Button(self.main_frame, text="City-wise Restaurant Count", command=self.query1).pack(fill='x')
        tk.Button(self.main_frame, text="Top 10 Restaurants by Avg Cost for 2", command=self.query2).pack(fill='x')
        tk.Button(self.main_frame, text="Top 10 Restaurants by Avg Votes", command=self.query3).pack(fill='x')
        tk.Button(self.main_frame, text="Top 10 Restaurants by Rating", command=self.query4).pack(fill='x')
        tk.Button(self.main_frame, text="Rating by Delivery Availability", command=self.query5).pack(fill='x')
        tk.Button(self.main_frame, text="Avg Cost for 2 by Cuisine", command=self.query6).pack(fill='x')

    def show_main_window(self):
        self.root.deiconify()

    def query1(self):
        self.root.withdraw()
        title = "City-wise Restaurant Count"
        sql_query = "SELECT City, COUNT(*) AS Count FROM swiggy_source GROUP BY City ORDER BY Count DESC"
        columns = ["City", "Count"]
        widths = [15, 10]
        self.execute_query(title, sql_query, columns, widths)

    def query2(self):
        self.root.withdraw()
        title = "Top 10 Restaurants by Avg Cost for 2"
        sql_query = "SELECT city, restaurant_name, avg_cost_for_two FROM swiggy_source ORDER BY avg_cost_for_two DESC LIMIT 10"
        columns = ["City", "Restaurant Name", "Avg Cost for 2"]
        widths = [15, 50, 15]
        self.execute_query(title, sql_query, columns, widths)

    def query3(self):
        self.root.withdraw()
        title = "Top 10 Restaurants by Avg Votes"
        sql_query = "SELECT City, Restaurant_Name, AVG(Votes) AS Votes FROM swiggy_source GROUP BY Restaurant_Name, City ORDER BY Votes DESC LIMIT 10"
        columns = ["City", "Restaurant Name", "Avg Votes"]
        widths = [15, 50, 15]
        self.execute_query(title, sql_query, columns, widths)

    def query4(self):
        self.root.withdraw()
        title = "Top 10 Restaurants by Rating"
        sql_query = "SELECT city City, restaurant_name `Restaurant Name`, rating_stars_out_of_5 Rating FROM swiggy_source ORDER BY city, Rating DESC LIMIT 10"
        columns = ["City", "Restaurant Name", "Rating"]
        widths = [15, 50, 10]
        self.execute_query(title, sql_query, columns, widths)

    def query5(self):
        self.root.withdraw()
        title = "Rating by Delivery Availability"
        sql_query = "SELECT City, has_online_delivery `Online Delivery`, rating_stars_out_of_5 Rating FROM swiggy_source"
        columns = ["City", "Online Delivery", "Rating"]
        widths = [15, 20, 10]
        self.execute_query(title, sql_query, columns, widths)

    def query6(self):
        self.root.withdraw()
        title = "Avg Cost for 2 by Cuisine"
        sql_query = "SELECT City, Cuisines Cuisine, avg_cost_for_two `Average Cost for two` FROM swiggy_source"
        columns = ["City", "Cuisine", "Average Cost for two"]
        widths = [15, 20, 20]
        self.execute_query(title, sql_query, columns, widths)

    def execute_query(self, title, sql_query, columns, widths):
        try:
            self.mycursor.execute(sql_query)
            result = self.mycursor.fetchall()
            ResultsWindow(self, title, columns, result, widths)
        except sql.Error as e:
            messagebox.showerror("Query Error", f"Failed to execute query: {e}")
            self.show_main_window()

    def close_connection(self):
        if hasattr(self, 'mydb') and self.mydb.is_connected():
            self.mydb.close()
            self.mycursor.close()

Output Window

In [3]:
class ResultsWindow(tk.Toplevel):
    def __init__(self, main_app, title, columns, data, widths):
        tk.Toplevel.__init__(self)
        self.main_app = main_app
        self.title(title)
        self.geometry("800x600")

        # Check if there are results to display
        if not data:
            tk.Label(self, text="No results found.").pack(pady=20)
        else:
            # Create a frame for the Treeview and scrollbar
            tree_frame = tk.Frame(self)
            tree_frame.pack(expand=True, fill='both')

            # Set up the Treeview
            tree = ttk.Treeview(tree_frame, columns=columns, show='headings')
            for col, width in zip(columns, widths):
                tree.heading(col, text=col)
                tree.column(col, width=width * 10)  # Convert characters to pixels (approx.)

            # Insert data into the Treeview
            for row in data:
                tree.insert('', 'end', values=row)

            # Add a vertical scrollbar
            scrollbar = tk.Scrollbar(tree_frame, orient="vertical", command=tree.yview)
            tree.configure(yscrollcommand=scrollbar.set)
            tree.pack(side='left', expand=True, fill='both')
            scrollbar.pack(side='right', fill='y')

        # Add a back button
        back_button = tk.Button(self, text="Back", command=self.go_back)
        back_button.pack(pady=10)

        # Handle window close event
        self.protocol("WM_DELETE_WINDOW", self.go_back)

    def go_back(self):
        self.destroy()
        self.main_app.show_main_window()

In [5]:
if __name__ == "__main__":
    root = tk.Tk()
    app = SwiggyApp(root)
    root.protocol("WM_DELETE_WINDOW", lambda: [app.close_connection(), root.destroy()])
    root.mainloop()